In [1]:
# warning filters
import warnings
warnings.filterwarnings("ignore", message="Pandas requires version")
warnings.filterwarnings("ignore", message="A NumPy version >=")

# general imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# anonymity library
import pycanon
from anjana.anonymity import k_anonymity, l_diversity, t_closeness

# ML models
from xgboost import XGBClassifier as XGB

# our generic functions
from utils import get_metrics, write_suppression_results_to_csv, get_generalization_levels, get_train_test_data

# our data-specific functions
from utils import clean_process_adult_data, get_hierarchies_adult
import config_experiments as cfg

# Main execution
write_suppression_results_to_csv([], header=True)

# Define the parameters
dataset = 'adult'

# Get parameters from config file
lst_supp_level = cfg.supp_level
dic_methods_parameters = {
                            'k-anonymity': cfg.fixed_k,
                            'l-diversity': cfg.fixed_l,
                            't-closeness': cfg.fixed_t
                        } 
max_seed = cfg.max_seed
test_size = cfg.test_size
if dataset == 'adult':
    lst_threshold_target = cfg.adult_threshold_target

# Loop over the three anonymization methods
for method, anon_parameter in dic_methods_parameters.items():
    print(f"Method: {method}, Parameter: {anon_parameter}") 

    # Loop over several threshold targets (same dataset but with different Y distribution)
    for threshold_target in lst_threshold_target:
        print(f"Threshold target: {threshold_target}")

        # read data
        if dataset == 'adult':

            # Sensitive/target and protected attributes
            sens_att = "income"
            protected_att = 'gender'
            
            # Read and process the data
            data = clean_process_adult_data(pd.read_csv("adult_reconstruction.csv"), sens_att, protected_att, threshold_target)

            # Import/defining the hierarquies for each quasi-identifier. 
            hierarchies = get_hierarchies_adult(data)

        # Define the quasi-identifiers and the sensitive/protected attribute
        quasi_ident = list(set(data.columns) - {protected_att} - {sens_att})

        # Loop over several seeds
        SEED = 0
        while SEED < max_seed:
            print(f"SEED: {SEED}")

            # Loop over several suppression values
            for supp_level in lst_supp_level:
                print(f"supp_level: {supp_level}")

                try:
                    # Split into train and test data
                    train_data, test_data = train_test_split(data, test_size=test_size, random_state=SEED)

                    # Anonymize data
                    train_data_anon = k_anonymity(train_data, [], quasi_ident, cfg.fixed_k, supp_level, hierarchies)
                    if 'index' in train_data_anon.columns:
                        del train_data_anon['index'] 

                    # Assert that the level of k-anonymity is at least k
                    actual_k_anonymity = pycanon.anonymity.k_anonymity(train_data_anon, quasi_ident)
                    assert actual_k_anonymity >= cfg.fixed_k, f"k-anonymity constraint not met: Expected >= {cfg.fixed_k}, but got {actual_k_anonymity}"

                    if method == 'l-diversity':
                        # Apply l-diversity
                        train_data_anon = l_diversity(train_data_anon, [], quasi_ident, sens_att, cfg.fixed_k, anon_parameter, supp_level, hierarchies)

                        # Assert that the level of l-diversity is exactly met
                        actual_l_diversity = pycanon.anonymity.l_diversity(train_data_anon, quasi_ident, [sens_att])
                        assert actual_l_diversity == anon_parameter, f"l-diversity constraint not met: Expected == {anon_parameter}, but got {actual_l_diversity}"

                    if method == 't-closeness':
                        # Apply t-closeness
                        train_data_anon = t_closeness(train_data_anon, [], quasi_ident, sens_att, cfg.fixed_k, anon_parameter, supp_level, hierarchies)

                        # Assert that the level of t-closeness is satisfied
                        actual_t_closeness = pycanon.anonymity.t_closeness(train_data_anon, quasi_ident, [sens_att], True)
                        assert actual_t_closeness <= anon_parameter, f"t-closeness constraint not met: Expected <= {anon_parameter}, but got {actual_t_closeness:.2f}"

                    # Get generalization levels of the training set to apply the same to the test set
                    generalization_levels = get_generalization_levels(train_data_anon, quasi_ident, hierarchies)

                    # Apply the same generalization levels to the test data (Except for the protected attribute: for fairness measurements)
                    for col in set(quasi_ident) - {protected_att}:
                        level = generalization_levels.get(col)
                        
                        if level is not None:
                            # Retrieve the mapping dictionary for this level
                            hierarchy_mapping = dict(zip(hierarchies[col][0], hierarchies[col][level]))
                            
                            # Apply the mapping to the test data
                            test_data[col] = test_data[col].map(hierarchy_mapping)

                    # Separate features and target
                    X_train, y_train, X_test, y_test = get_train_test_data(train_data_anon, test_data, sens_att)

                    # Train the model
                    model = XGB(random_state=SEED, n_jobs=-1)
                    model.fit(X_train, y_train)

                    # Get fairness/utility metrics
                    df_fm = test_data.copy()
                    df_fm['y_pred'] = np.round(model.predict(X_test)).reshape(-1).astype(int)
                    dic_metrics = get_metrics(df_fm, protected_att, sens_att)
                    print(dic_metrics)

                    # Write results to csv
                    write_suppression_results_to_csv([SEED, dataset + "_" + str(threshold_target), protected_att, sens_att, method, anon_parameter, supp_level] + list(dic_metrics.values()))

                except Exception as e:
                        print(f"An error occurred for SEED {SEED}, k {cfg.fixed_k}: {e}")
                        continue
            
            SEED += 1
            print('-------------------------------------------------------------\n')
        print('=============================================================\n')
    print('#############################################################\n')

Method: k-anonymity, Parameter: 10
Threshold target: 10000
SEED: 0
supp_level: 10
{'SPD': 0.09984501563726589, 'EOD': 0.03099627356838386, 'MAD': 0.10314323898678057, 'PED': 0.059362139917695456, 'PRD': 0.11007519667192844, 'ACC': 0.8740821519447474, 'f1': 0.9269136636003038, 'Precision': 0.8818853380440019, 'Recall': 0.9767876200640342, 'ROC_AUC': 0.6954826192713639, 'CM': array([[ 1040,  1471],
       [  261, 10983]], dtype=int64)}
supp_level: 20
{'SPD': 0.11376717725554963, 'EOD': 0.04605595084701397, 'MAD': 0.108468011029481, 'PED': 0.050565843621399154, 'PRD': 0.10891484480250502, 'ACC': 0.8724827335514358, 'f1': 0.9253235694822888, 'Precision': 0.8875367526951976, 'Recall': 0.9664710067591604, 'ROC_AUC': 0.7090419549924835, 'CM': array([[ 1134,  1377],
       [  377, 10867]], dtype=int64)}
supp_level: 30
{'SPD': 0.09480889760139277, 'EOD': 0.030131841327606046, 'MAD': 0.10530485532883505, 'PED': 0.028755144032921898, 'PRD': 0.1139204175466565, 'ACC': 0.8732824427480916, 'f1': 0.9

c:\Users\hhwangar\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'SPD': 0.12677864555472343, 'EOD': 0.22633883462445267, 'MAD': -0.18765481203922874, 'PED': 0.08043273753527752, 'PRD': 0.5670886075949367, 'ACC': 0.7595056343147947, 'f1': 0.2889079965606191, 'Precision': 0.5670886075949367, 'Recall': 0.19382751658494377, 'ROC_AUC': 0.5719817987279306, 'CM': array([[9775,  513],
       [2795,  672]], dtype=int64)}
-------------------------------------------------------------

SEED: 1
supp_level: 10
The data verifies k-anonymity with k=10
{'SPD': 0.11496784592818982, 'EOD': 0.08699040864902377, 'MAD': -0.1749109680040638, 'PED': 0.0754350063742841, 'PRD': -0.000733477748641409, 'ACC': 0.7704834605597964, 'f1': 0.3774403470715835, 'Precision': 0.5803517283201941, 'Recall': 0.2796610169491525, 'ROC_AUC': 0.6063455573471205, 'CM': array([[9641,  692],
       [2465,  957]], dtype=int64)}
supp_level: 20
The data verifies k-anonymity with k=10
{'SPD': 0.1250797405519617, 'EOD': 0.10484067730444543, 'MAD': -0.17701847201690102, 'PED': 0.08363034255518281, 'P

c:\Users\hhwangar\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'SPD': 0.267027027027027, 'EOD': 0.49592053919829726, 'MAD': -0.15564003959564443, 'PED': 0.16669258280205257, 'PRD': 0.5659919028340081, 'ACC': 0.7814612868047982, 'f1': 0.4819027921406412, 'Precision': 0.5659919028340081, 'Recall': 0.41956782713085233, 'ROC_AUC': 0.6583591798035533, 'CM': array([[9351, 1072],
       [1934, 1398]], dtype=int64)}
-------------------------------------------------------------

SEED: 3
supp_level: 10
The data verifies k-anonymity with k=10
{'SPD': 0.21777742800102634, 'EOD': 0.16019311429570082, 'MAD': -0.14323208708754176, 'PED': 0.12879868083693988, 'PRD': -0.034276469014828836, 'ACC': 0.813595056343148, 'f1': 0.6013681592039801, 'Precision': 0.6322327558025499, 'Recall': 0.5733768158908983, 'ROC_AUC': 0.7325080958668515, 'CM': array([[9257, 1125],
       [1439, 1934]], dtype=int64)}
supp_level: 20
The data verifies k-anonymity with k=10
{'SPD': 0.1596716479038296, 'EOD': 0.017077043647326096, 'MAD': -0.13928811881936576, 'PED': 0.08868125158562701, 'P

...

{'SPD': 0.31541373715521903, 'EOD': 0.5557503506311361, 'MAD': -0.1650522425377654, 'PED': 0.2081964648834663, 'PRD': 0.5435528120713305, 'ACC': 0.7731006906579425, 'f1': 0.5038944523923063, 'Precision': 0.5435528120713305, 'Recall': 0.4696296296296296, 'ROC_AUC': 0.6707011346606722, 'CM': array([[9049, 1331],
       [1790, 1585]], dtype=int64)}
-------------------------------------------------------------

SEED: 94
supp_level: 10
The data verifies k-anonymity with k=10
{'SPD': 0.1689021392402868, 'EOD': 0.02898743992169195, 'MAD': -0.14586103932132755, 'PED': 0.09469639327945026, 'PRD': 0.02895803638823502, 'ACC': 0.8101781170483461, 'f1': 0.5850945494994438, 'Precision': 0.6428072625698324, 'Recall': 0.536891221930592, 'ROC_AUC': 0.718910456985052, 'CM': array([[9303, 1023],
       [1588, 1841]], dtype=int64)}
supp_level: 20
The data verifies k-anonymity with k=10
{'SPD': 0.1689021392402868, 'EOD': 0.02898743992169195, 'MAD': -0.14586103932132755, 'PED': 0.09469639327945026, 'PRD': 0

c:\Users\hhwangar\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'SPD': 0.3443609022556391, 'EOD': 0.5953375086986779, 'MAD': -0.16693928296556915, 'PED': 0.23228713486637664, 'PRD': 0.533686837180287, 'ACC': 0.7684478371501272, 'f1': 0.5179355229302255, 'Precision': 0.533686837180287, 'Recall': 0.5030873272566893, 'ROC_AUC': 0.679349342592996, 'CM': array([[8859, 1495],
       [1690, 1711]], dtype=int64)}
supp_level: 50
The data verifies k-anonymity with k=10


c:\Users\hhwangar\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'SPD': 0.3374865735767991, 'EOD': 0.5852470424495476, 'MAD': -0.16629481465192797, 'PED': 0.22684897451833436, 'PRD': 0.5353278166772756, 'ACC': 0.7688840421664849, 'f1': 0.5141372459116613, 'Precision': 0.5353278166772756, 'Recall': 0.49456042340488093, 'ROC_AUC': 0.6767760587180865, 'CM': array([[8894, 1460],
       [1719, 1682]], dtype=int64)}
-------------------------------------------------------------

SEED: 98
supp_level: 10
The data verifies k-anonymity with k=10
{'SPD': 0.10228717354856584, 'EOD': -0.03966713647000114, 'MAD': -0.17816273315020403, 'PED': 0.07459829711886248, 'PRD': 0.04801714963451975, 'ACC': 0.7653944020356234, 'f1': 0.4144438395935402, 'Precision': 0.540719696969697, 'Recall': 0.33598117093262725, 'ROC_AUC': 0.621157831507256, 'CM': array([[9386,  970],
       [2257, 1142]], dtype=int64)}
supp_level: 20
The data verifies k-anonymity with k=10
{'SPD': 0.11032109454834269, 'EOD': -0.006593206508911653, 'MAD': -0.178609062094636, 'PED': 0.07938781060512413, 'P